# Cognitive Searchのベクトル検索
1. ナレッジ登録
2. ベクトル検索

In [1]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from azure.search.documents.indexes.models import (
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField
)

load_dotenv()

# Azure OpenAIのエンドポイントとAPIキー
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")
# Embeddingモデルのデプロイ名
AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME=os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME")
# Azure SearchのエンドポイントとAPIキー
AZURE_SEARCH_ENDPOINT=os.getenv("AZURE_SEARCH_ENDPOINT")
AZURE_SEARCH_SERVICE_NAME=os.getenv("AZURE_SEARCH_SERVICE_NAME")
AZURE_SEARCH_API_KEY_ADMIN=os.getenv("AZURE_SEARCH_API_KEY_ADMIN")


## PDFファイルの分割
- [LangChain -> PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)
- [LangChain -> Text Splitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/)

In [2]:
# PDFを読み込んで、テキストを分割する
loader = PyPDFLoader("../data/Azure OpenAI Service とは - Azure Cognitive Services _ Microsoft Learn.pdf")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# 分割後のドキュメントを表示
for doc in docs:
    print(doc.json(indent=2, ensure_ascii=False))

{
  "page_content": "Azure OpenAI Service とは\n[アーティクル ]•2023/05/01\nAzure OpenAI Service では、 GPT-3 、 Codex 、 Embeddings モデル  シリーズなど\nOpenAI の強⼒な⾔語モデルを  REST API として使⽤できます。  さらに、新しい  GPT-4\nおよび  ChatGPT (gpt-35-turbo) モデル  シリーズがプレビューで利⽤可能になりまし\nた。 これらのモデルは、特定のタスクに合わせて簡単に調整できます。たとえば、コ\nンテンツの⽣成、まとめ、セマンティック検索、⾃然⾔語からコードへの翻訳などで\nす。 ユーザーは、 REST API 、 Python SDK 、または  Azure OpenAI S tudio の  Web ベース\nのインターフェイスを介してサービスにアクセスできます。\n機能 Azur e OpenAI\n使⽤できるモデル 新しい  GPT-4 シリーズ  ( プレビュー )\nGPT-3 ベース  シリーズ\n新しい  ChatGPT (gpt -35-turbo) ( プレビュー )\nCodex シリーズ\n埋め込みシリーズ\n詳細については、 モデルに関するページを参照してください。\n微調整 Ada\nBabbage\nCurie\nCushman*\nDavinci*\n* 現在は利⽤できません。  ** ⽶国東部と⻄ヨーロッパでは、現在新\n規のお客様は微調整を利⽤できません。  ⽶国ベースのトレーニング\nには、⽶国中南部をご利⽤ください\nPrice こちらで⼊⼿可能\n仮想ネットワークのサポ\nート & プライベート  リン\nクのサポートはい\nマネージド  ID はい、 Azure Active Directory 経由\nUI エクスペリエンス アカウントとリソースの管理には  Azur e Portal、\nモデルの探索と微調整には  Azur e OpenAI Ser vice S tudio\nリージョン別の提供状況 ⽶国東部\n⽶国中南部\n⻄ヨーロッパ機能の概要",
  "metadata": {
    "source": "../data/Azure Open

## Cognitive Searchのインデックス作成
- プレビュー版(azure-search-documents==11.4.0b6)のSDKを利用
- https://python.langchain.com/docs/integrations/vectorstores/azuresearch
- https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.azuresearch.AzureSearch.html


In [3]:
# emmbeddingモデルを定義
embeddings = OpenAIEmbeddings(
    openai_api_type="azure",
    model='text-embedding-ada-002',
    openai_api_base=AZURE_OPENAI_ENDPOINT,
    openai_api_key=AZURE_OPENAI_API_KEY,
    deployment=AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME)

# Cognitive Searchのインデックスを作成
fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
        analyzer_name="ja.lucene",
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=len(embeddings.embed_query("Text")),
        vector_search_configuration="default",
        
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
]

# インデックス名
index_name = "fixed-sample-index"

# ベクトルストアを作成
vector_store = AzureSearch(
    azure_search_endpoint=AZURE_SEARCH_ENDPOINT,
    azure_search_key=AZURE_SEARCH_API_KEY_ADMIN,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    fields=fields
)

# ベクトルDBに追加
vector_store.add_documents(documents=docs)

['MWU1YWQxMjQtN2Y4NC00YTRhLWI1MjEtYzU3MWQ4Mzc4ODU4',
 'YmI4ODI0ZGEtYTkzYi00OTBkLTliNjItNDdmNGRjMTVkMThl',
 'Y2M3MDBmNmItZmM2My00YTU4LTk0MTktZDU3ZGJlOWJkZDMz',
 'NGMzNDBjMTItZDUzYy00NjdhLWI2Y2QtYjJiODk0N2FiM2Fm',
 'MDVlNTk0MDQtOWI5Mi00NmJkLWEyZjctYzZkZjllMjEyZDE5']

## ベクトル検索

In [4]:
result_docs = vector_store.similarity_search(
    query="Azure OpenAIで使えるモデルを教えて",
    k=3,
    search_type="similarity"
    )

for doc in result_docs:
    print(doc.json(indent=2, ensure_ascii=False))

{
  "page_content": "Azure OpenAI Service とは\n[アーティクル ]•2023/05/01\nAzure OpenAI Service では、 GPT-3 、 Codex 、 Embeddings モデル  シリーズなど\nOpenAI の強⼒な⾔語モデルを  REST API として使⽤できます。  さらに、新しい  GPT-4\nおよび  ChatGPT (gpt-35-turbo) モデル  シリーズがプレビューで利⽤可能になりまし\nた。 これらのモデルは、特定のタスクに合わせて簡単に調整できます。たとえば、コ\nンテンツの⽣成、まとめ、セマンティック検索、⾃然⾔語からコードへの翻訳などで\nす。 ユーザーは、 REST API 、 Python SDK 、または  Azure OpenAI S tudio の  Web ベース\nのインターフェイスを介してサービスにアクセスできます。\n機能 Azur e OpenAI\n使⽤できるモデル 新しい  GPT-4 シリーズ  ( プレビュー )\nGPT-3 ベース  シリーズ\n新しい  ChatGPT (gpt -35-turbo) ( プレビュー )\nCodex シリーズ\n埋め込みシリーズ\n詳細については、 モデルに関するページを参照してください。\n微調整 Ada\nBabbage\nCurie\nCushman*\nDavinci*\n* 現在は利⽤できません。  ** ⽶国東部と⻄ヨーロッパでは、現在新\n規のお客様は微調整を利⽤できません。  ⽶国ベースのトレーニング\nには、⽶国中南部をご利⽤ください\nPrice こちらで⼊⼿可能\n仮想ネットワークのサポ\nート & プライベート  リン\nクのサポートはい\nマネージド  ID はい、 Azure Active Directory 経由\nUI エクスペリエンス アカウントとリソースの管理には  Azur e Portal、\nモデルの探索と微調整には  Azur e OpenAI Ser vice S tudio\nリージョン別の提供状況 ⽶国東部\n⽶国中南部\n⻄ヨーロッパ機能の概要",
  "metadata": {
    "source": "../data/Azure Open

## 検索スコア付きベクトル検索

In [5]:
result_docs = vector_store.vector_search_with_score(
    query="Azure OpenAIで使えるモデルを教えて",
    k=3
    )

display(result_docs)

[(Document(page_content='Azure OpenAI Service とは\n[アーティクル ]•2023/05/01\nAzure OpenAI Service では、 GPT-3 、 Codex 、 Embeddings モデル  シリーズなど\nOpenAI の強⼒な⾔語モデルを  REST API として使⽤できます。  さらに、新しい  GPT-4\nおよび  ChatGPT (gpt-35-turbo) モデル  シリーズがプレビューで利⽤可能になりまし\nた。 これらのモデルは、特定のタスクに合わせて簡単に調整できます。たとえば、コ\nンテンツの⽣成、まとめ、セマンティック検索、⾃然⾔語からコードへの翻訳などで\nす。 ユーザーは、 REST API 、 Python SDK 、または  Azure OpenAI S tudio の  Web ベース\nのインターフェイスを介してサービスにアクセスできます。\n機能 Azur e OpenAI\n使⽤できるモデル 新しい  GPT-4 シリーズ  ( プレビュー )\nGPT-3 ベース  シリーズ\n新しい  ChatGPT (gpt -35-turbo) ( プレビュー )\nCodex シリーズ\n埋め込みシリーズ\n詳細については、 モデルに関するページを参照してください。\n微調整 Ada\nBabbage\nCurie\nCushman*\nDavinci*\n* 現在は利⽤できません。  ** ⽶国東部と⻄ヨーロッパでは、現在新\n規のお客様は微調整を利⽤できません。  ⽶国ベースのトレーニング\nには、⽶国中南部をご利⽤ください\nPrice こちらで⼊⼿可能\n仮想ネットワークのサポ\nート & プライベート  リン\nクのサポートはい\nマネージド  ID はい、 Azure Active Directory 経由\nUI エクスペリエンス アカウントとリソースの管理には  Azur e Portal、\nモデルの探索と微調整には  Azur e OpenAI Ser vice S tudio\nリージョン別の提供状況 ⽶国東部\n⽶国中南部\n⻄ヨーロッパ機能の概要', metadata={'source': '../data/Azure OpenAI Ser

## ハイブリッド検索

In [6]:
result_docs = vector_store.similarity_search(
    query="Azure OpenAIで使えるモデルを教えて",
    k=3,
    search_type="hybrid"
)

for doc in result_docs:
    print(doc.json(indent=2, ensure_ascii=False))

{
  "page_content": "Azure OpenAI Service とは\n[アーティクル ]•2023/05/01\nAzure OpenAI Service では、 GPT-3 、 Codex 、 Embeddings モデル  シリーズなど\nOpenAI の強⼒な⾔語モデルを  REST API として使⽤できます。  さらに、新しい  GPT-4\nおよび  ChatGPT (gpt-35-turbo) モデル  シリーズがプレビューで利⽤可能になりまし\nた。 これらのモデルは、特定のタスクに合わせて簡単に調整できます。たとえば、コ\nンテンツの⽣成、まとめ、セマンティック検索、⾃然⾔語からコードへの翻訳などで\nす。 ユーザーは、 REST API 、 Python SDK 、または  Azure OpenAI S tudio の  Web ベース\nのインターフェイスを介してサービスにアクセスできます。\n機能 Azur e OpenAI\n使⽤できるモデル 新しい  GPT-4 シリーズ  ( プレビュー )\nGPT-3 ベース  シリーズ\n新しい  ChatGPT (gpt -35-turbo) ( プレビュー )\nCodex シリーズ\n埋め込みシリーズ\n詳細については、 モデルに関するページを参照してください。\n微調整 Ada\nBabbage\nCurie\nCushman*\nDavinci*\n* 現在は利⽤できません。  ** ⽶国東部と⻄ヨーロッパでは、現在新\n規のお客様は微調整を利⽤できません。  ⽶国ベースのトレーニング\nには、⽶国中南部をご利⽤ください\nPrice こちらで⼊⼿可能\n仮想ネットワークのサポ\nート & プライベート  リン\nクのサポートはい\nマネージド  ID はい、 Azure Active Directory 経由\nUI エクスペリエンス アカウントとリソースの管理には  Azur e Portal、\nモデルの探索と微調整には  Azur e OpenAI Ser vice S tudio\nリージョン別の提供状況 ⽶国東部\n⽶国中南部\n⻄ヨーロッパ機能の概要",
  "metadata": {
    "source": "../data/Azure Open

## 検索スコア付きハイブリッド検索

In [7]:
result_docs = vector_store.hybrid_search_with_score(
    query="Azure OpenAIで使えるモデルを教えて",
    k=3
    )

display(result_docs)

[(Document(page_content='Azure OpenAI Service とは\n[アーティクル ]•2023/05/01\nAzure OpenAI Service では、 GPT-3 、 Codex 、 Embeddings モデル  シリーズなど\nOpenAI の強⼒な⾔語モデルを  REST API として使⽤できます。  さらに、新しい  GPT-4\nおよび  ChatGPT (gpt-35-turbo) モデル  シリーズがプレビューで利⽤可能になりまし\nた。 これらのモデルは、特定のタスクに合わせて簡単に調整できます。たとえば、コ\nンテンツの⽣成、まとめ、セマンティック検索、⾃然⾔語からコードへの翻訳などで\nす。 ユーザーは、 REST API 、 Python SDK 、または  Azure OpenAI S tudio の  Web ベース\nのインターフェイスを介してサービスにアクセスできます。\n機能 Azur e OpenAI\n使⽤できるモデル 新しい  GPT-4 シリーズ  ( プレビュー )\nGPT-3 ベース  シリーズ\n新しい  ChatGPT (gpt -35-turbo) ( プレビュー )\nCodex シリーズ\n埋め込みシリーズ\n詳細については、 モデルに関するページを参照してください。\n微調整 Ada\nBabbage\nCurie\nCushman*\nDavinci*\n* 現在は利⽤できません。  ** ⽶国東部と⻄ヨーロッパでは、現在新\n規のお客様は微調整を利⽤できません。  ⽶国ベースのトレーニング\nには、⽶国中南部をご利⽤ください\nPrice こちらで⼊⼿可能\n仮想ネットワークのサポ\nート & プライベート  リン\nクのサポートはい\nマネージド  ID はい、 Azure Active Directory 経由\nUI エクスペリエンス アカウントとリソースの管理には  Azur e Portal、\nモデルの探索と微調整には  Azur e OpenAI Ser vice S tudio\nリージョン別の提供状況 ⽶国東部\n⽶国中南部\n⻄ヨーロッパ機能の概要', metadata={'source': '../data/Azure OpenAI Ser

## Azure SDKによる検索
- azure-search-documents==11.4.0b8

In [8]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import Vector
import numpy as np

search_client = SearchClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    index_name=index_name,
    credential=AzureKeyCredential(AZURE_SEARCH_API_KEY_ADMIN),
)

query = "Azure OpenAIで使えるモデルを教えて"

# ベクトルを作成
vector = Vector(
    value=np.array(
        embeddings.embed_query(query),
        dtype=np.float32).tolist(),
        k=3,
        fields="content_vector",
        )

results = search_client.search(
    search_text="", vectors=[vector]
)

for result in results:
    # print(json.dumps(result, indent=2, sort_keys=True))
    print("###############")
    print(f"ID: {result['id']}")
    print(f"Score: {result['@search.score']}")
    print(f"Metadata: {result['metadata']}")
    print(f"Content: {result['content']}")

###############
ID: MWU1YWQxMjQtN2Y4NC00YTRhLWI1MjEtYzU3MWQ4Mzc4ODU4
Score: 0.9002142
Metadata: {"source": "../data/Azure OpenAI Service \u3068\u306f - Azure Cognitive Services _ Microsoft Learn.pdf", "page": 0}
Content: Azure OpenAI Service とは
[アーティクル ]•2023/05/01
Azure OpenAI Service では、 GPT-3 、 Codex 、 Embeddings モデル  シリーズなど
OpenAI の強⼒な⾔語モデルを  REST API として使⽤できます。  さらに、新しい  GPT-4
および  ChatGPT (gpt-35-turbo) モデル  シリーズがプレビューで利⽤可能になりまし
た。 これらのモデルは、特定のタスクに合わせて簡単に調整できます。たとえば、コ
ンテンツの⽣成、まとめ、セマンティック検索、⾃然⾔語からコードへの翻訳などで
す。 ユーザーは、 REST API 、 Python SDK 、または  Azure OpenAI S tudio の  Web ベース
のインターフェイスを介してサービスにアクセスできます。
機能 Azur e OpenAI
使⽤できるモデル 新しい  GPT-4 シリーズ  ( プレビュー )
GPT-3 ベース  シリーズ
新しい  ChatGPT (gpt -35-turbo) ( プレビュー )
Codex シリーズ
埋め込みシリーズ
詳細については、 モデルに関するページを参照してください。
微調整 Ada
Babbage
Curie
Cushman*
Davinci*
* 現在は利⽤できません。  ** ⽶国東部と⻄ヨーロッパでは、現在新
規のお客様は微調整を利⽤できません。  ⽶国ベースのトレーニング
には、⽶国中南部をご利⽤ください
Price こちらで⼊⼿可能
仮想ネットワークのサポ
ート & プライベート  リン
クのサポートはい
マネージド  ID はい、 Azure Active Directory 経由
UI エクスペリ

## インデックスの削除

In [9]:
from azure.search.documents.indexes import SearchIndexClient

search_index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    credential=AzureKeyCredential(AZURE_SEARCH_API_KEY_ADMIN)
    )

search_index_client.delete_index(index_name)